In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import numpy as np

In [25]:
model = models.resnet18(pretrained=True)
print(model.fc.in_features)
model.fc = nn.Linear(model.fc.in_features, 128)

512


In [16]:
x_3d = torch.randn((32, 4, 3, 144, 144))
x_3d.shape

torch.Size([32, 4, 3, 144, 144])

In [19]:
outputs = []
for t in range(x_3d.size(1)):  # time 크기
    x = x_3d[:, t, :, :, :]  # (batch, channel, x, y)
    print(f"x: {x.shape}")
    x = model(x)
    print(f"model output: {x.shape}")
    outputs.append(x)
outputs = torch.stack(outputs, 0).transpose_(0,1)
print(f"outputs: {outputs.shape}")

x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
outputs: torch.Size([32, 4, 128])


In [18]:
outputs = []
for t in range(x_3d.size(1)):  # time 크기
    x = x_3d[:, t, :, :, :]  # (batch, channel, x, y)
    x = model(x)
    outputs.append(x)
outputs = torch.stack(outputs, 1)

x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
x: torch.Size([32, 3, 144, 144])
model output: torch.Size([32, 128])
outputs: torch.Size([32, 4, 128])


In [27]:
class R2Plus1dStem(nn.Sequential):
    """R(2+1)D stem is different than the default one as it uses separated 3D convolution
    """
    def __init__(self):
        super(R2Plus1dStem, self).__init__(
            nn.Conv3d(3, 45, kernel_size=(1, 7, 7),
                      stride=(1, 2, 2), padding=(0, 3, 3),
                      bias=False),
            nn.BatchNorm3d(45),
            nn.ReLU(inplace=True),
            nn.Conv3d(45, 64, kernel_size=(3, 1, 1),
                      stride=(1, 1, 1), padding=(1, 0, 0),
                      bias=False),
            nn.BatchNorm3d(64),
            nn.ReLU(inplace=True))

In [31]:
R2Plus1dStem()(torch.randn((1, 3, 5, 120, 120))).shape

torch.Size([1, 64, 5, 60, 60])

In [46]:
model = models.video.r2plus1d_18(True)
for param in model.parameters():
    param.requires_grad = False
model.eval()
print(model.fc.in_features)
model.fc = nn.Linear(model_vid.fc.in_features, 128)

512


In [62]:
np.array(list(map(lambda x: x.detach().numpy(), filter(lambda p: p.requires_grad, model_vid.parameters()))))

ValueError: could not broadcast input array from shape (128,512) into shape (128)